CIS 600 Applied Agetic AI Systems
Spring 2026

Assignment II


In [31]:
!pip install langgraph langchain langchain-community langchain-core

In [32]:
!sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [33]:
import subprocess, time

subprocess.Popen(["ollama", "serve"])
time.sleep(5)
result = subprocess.run(["ollama", "pull", "llama3.2"], capture_output=False)
print("Done!", result.returncode)

Done! 0


In [34]:
import requests

res = requests.post("http://localhost:11434/api/generate",
    json={"model": "llama3.2", "prompt": "Say hello", "stream": False})
print(res.json()["response"])

Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


* First, the required Python libraries were installed such as langgraph, langchain, langchain-community, and langchain-core since Colab doesn't have these by default.
* Ollama couldn't be installed straight away because the newer version needs zstd for extraction, so that was installed first using apt-get before re-running the Ollama install script.
* Once Ollama was installed, the server was started in the background using Python's subprocess module, and the mistral model was pulled since that is the default LLM mentioned in the template and what will be used throughout this assignment.
* Finally, a quick test was run by sending a prompt to the local Ollama API endpoint to confirm the server was up and the model was responding correctly before writing any agent code.

In [35]:
subprocess.run(["ollama", "pull", "mistral"], capture_output=False)

CompletedProcess(args=['ollama', 'pull', 'mistral'], returncode=0)

In [36]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama
# OR use HuggingFaceHub

In [37]:

#################################################
# TODO 1: Initialize Your Free LLM
#################################################

# Option A: Ollama (recommended if installed locally)
llm = ChatOllama(model="mistral")


Initializes the free LLM using ChatOllama with the Mistral model running locally through Ollama. This is the core language model that powers all decision-making and answer generation in the agent.

In [38]:

#################################################
# TODO 2: Define Agent State
#################################################

class AgentState(TypedDict):
    question: str
    decision: Optional[str]
    tool_input: Optional[str]
    tool_output: Optional[str]
    final_answer: Optional[str]



Defines the AgentState TypedDict which acts as shared memory across all nodes.

It holds the question, decision, tool input/output, and final answer throughout the workflow.

In [39]:
#################################################
# TODO 3: Create a Tool
#################################################

@tool
def calculator(expression: str) -> str:
    """
    Evaluate a basic mathematical expression.
    """
    try:
        return str(eval(expression))
    except Exception:
        return "Error in calculation."


Creates a calculator tool using the @tool decorator that evaluates a mathematical expression using Python's eval().

It returns the result as a string or an error message if the expression is invalid.

In [40]:
#################################################
# TODO 4: Decision Node
#################################################

def decision_node(state: AgentState) -> AgentState:
    """
    Decide whether to use tool or answer directly.
    """
    question = state["question"]

    prompt = f"""
    You are a decision-making agent.
    If the question requires math calculation, output: use_tool
    Otherwise output: no_tool

    Question: {question}
    """

    response = llm.invoke([HumanMessage(content=prompt)])

    # Normalize decision to avoid fragile exact string matching
    raw = response.content.strip().lower()
    if "use_tool" in raw:
        decision = "use_tool"
    else:
        decision = "no_tool"

    return {
        **state,
        "decision": decision
    }

The decision node prompts the LLM to decide whether the question needs the calculator tool or can be answered directly.

The raw response is normalized using a lowercase check with "in" instead of exact string matching to handle cases where Mistral returns extra words alongside the keyword, making routing more robust.

In [41]:
#################################################
# TODO 5: Tool Node
#################################################

import re

def tool_node(state: AgentState) -> AgentState:
    """
    Executes the calculator tool.
    """
    question = state["question"]

    # Remove everything except valid math characters
    expression = re.sub(r"[^\d\.\+\-\*\/\(\)]", "", question)

    result = calculator.invoke(expression)

    return {
        **state,
        "tool_input": expression,
        "tool_output": result
    }

The tool node extracts the math expression from the question using regex and passes it to the calculator tool.

The result is stored back into the state as tool_output.

The template directly passed the full question string to calculator.invoke(question), which caused an "Error in calculation".

This was fixed by adding regex extraction to pull only the mathematical expression before passing it to the calculator.

In [42]:
#################################################
# TODO 6: Final Answer Node
#################################################

def answer_node(state: AgentState) -> AgentState:
    """
    Generate final answer.
    """

    question = state["question"]
    tool_output = state.get("tool_output", "")

    # TODO:
    # If tool_output exists, use it in final answer.
    # Otherwise answer directly with LLM.

    if tool_output:
        final_answer = f"The result is: {tool_output}"
    else:
        response = llm.invoke([HumanMessage(content=question)])
        final_answer = response.content

    return {
        **state,
        "final_answer": final_answer
    }


The answer node checks whether a tool_output exists and formats the final response accordingly.

If no tool was used, it calls the LLM directly to generate an answer

In [43]:
#################################################
# TODO 7: Conditional Routing Function
#################################################

def route_decision(state: AgentState):
    """
    Route based on decision.
    """
    if state["decision"] == "use_tool":
        return "tool_node"
    else:
        return "answer_node"


The routing function reads the decision from state and returns either "tool_node" or "answer_node" as the next step. This is what drives the conditional branching in the graph.

In [44]:
#################################################
# TODO 8: Build the Graph
#################################################

workflow = StateGraph(AgentState)

workflow.add_node("decision_node", decision_node)
workflow.add_node("tool_node", tool_node)
workflow.add_node("answer_node", answer_node)

workflow.set_entry_point("decision_node")

workflow.add_conditional_edges(
    "decision_node",
    route_decision,
    {
        "tool_node": "tool_node",
        "answer_node": "answer_node"
    }
)

workflow.add_edge("tool_node", "answer_node")
workflow.add_edge("answer_node", END)

app = workflow.compile()



 Builds and compiles the full LangGraph workflow by registering all nodes, setting the entry point, and wiring the conditional and regular edges together.

In [45]:
#################################################
# TODO 9: Run the Agent
#################################################

# Test 1: Math question uses calculator tool
result1 = app.invoke({
    "question": "What is 39 * 4 + 10?",
    "decision": None,
    "tool_input": None,
    "tool_output": None,
    "final_answer": None
})
print("Test 1 - Math Question:")
print(result1["final_answer"])

# Test 2: General question which answers directly
result2 = app.invoke({
    "question": "What is the Capital of India?",
    "decision": None,
    "tool_input": None,
    "tool_output": None,
    "final_answer": None
})
print("Test 2 - General Question:")
print(result2["final_answer"])

Test 1 - Math Question:
The result is: 166
Test 2 - General Question:
 The capital of India is New Delhi. It serves as both a city and a union territory in its own right, and it's divided into two parts: Old Delhi (North) and New Delhi (South). New Delhi is known for landmarks that reflect Indian history and modernization like the Red Fort, Qutub Minar, Humayun's Tomb, India Gate, and the Parliament House.


Runs two test cases through the compiled agent
* one math question
* one general question

To verify both routing paths work correctly.

The template used input() to accept questions from the user at runtime, which does not work well in Google Colab.
So I replaced it with two hardcoded test questions one math question and one general question to demonstrate both routing paths of the agent clearly.

Tools used:



* LLM: Mistral through ChatOllama
* Framework: LangGraph StateGraph
* Tool: Calculator
* Nodes: decision_node, tool_node, answer_node
* Conditional Edge: decision_node to tool_node or answer_node
* State: AgentState TypedDict
* Deliverables: .ipynb with outputs and reflection

**Why use LangGraph instead of a simple agent?**

When I first started building this, a simple LLM call felt enough. You send a question, you get an answer. But the problem is that a basic LLM call has no memory of what happened before it and no structure to follow. LangGraph changes that by letting it define the workflow as a graph where each node has a specific job and the state carries information from one step to the next. The other reason is conditional routing. With a plain LLM call it would have to write our own logic outside the model to decide what happens next. LangGraph handles that natively through conditional edges which makes the whole flow cleaner and easier to debug.

**What limitations did you observe?**

The biggest issue I ran into was with the decision node. Mistral sometimes returned a full sentence instead of just use_tool or no_tool which broke the routing. This was fixed by normalizing the response using a lowercase "in" check instead of exact string matching. The second limitation was speed  running Mistral on CPU in Colab with no GPU meant every LLM call took a couple of minutes, which would be a serious bottleneck in any real application.

